<a href="https://colab.research.google.com/github/matthewpecsok/data_engineering/blob/main/tutorials/de_uta_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Get a UTA token

https://developer.rideuta.com/Registration.aspx

In [76]:
#!pip install xmltodict

In [77]:
from google.colab import userdata
import xmltodict

uta_secret = userdata.get('uta')

In [78]:
route = "703"

In [79]:
import requests

url = f'http://api.rideuta.com/SIRI/SIRI.svc/VehicleMonitor/ByRoute?route={route}&onwardcalls=true&usertoken={uta_secret}'
response = requests.get(url)
print(response.text)

xml_dict = xmltodict.parse(response.text)

<?xml version="1.0" encoding="utf-8"?><Siri version="1.3" xmlns="http://www.siri.org.uk/siri"><ResponseTimestamp>2024-03-05T16:52:21.601903-07:00</ResponseTimestamp><VehicleMonitoringDelivery version="1.3"><ResponseTimestamp>2024-03-05T16:52:21.601903-07:00</ResponseTimestamp><ValidUntil>2024-03-05T16:52:31.601903-07:00</ValidUntil><VehicleActivity><RecordedAtTime>2024-03-05T16:52:21.601903-07:00</RecordedAtTime><MonitoredVehicleJourney><LineRef>703</LineRef><DirectionRef/><FramedVehicleJourneyRef><DataFrameRef>2024-03-05T00:00:00-07:00</DataFrameRef><DatedVehicleJourneyRef>0</DatedVehicleJourneyRef></FramedVehicleJourneyRef><PublishedLineName>RED LINE</PublishedLineName><OriginRef>0</OriginRef><DestinationRef>0</DestinationRef><Monitored>True</Monitored><VehicleLocation><Longitude>-111.890983</Longitude><Latitude>40.758957</Latitude></VehicleLocation><ProgressRate>1</ProgressRate><CourseOfJourneyRef>400864</CourseOfJourneyRef><VehicleRef>1102</VehicleRef><Extensions><LastGPSFix>2024-0

In [ ]:
xml_dict['Siri']['VehicleMonitoringDelivery']['VehicleActivity']['MonitoredVehicleJourney']

In [81]:
import pandas as pd

df = pd.DataFrame(xml_dict['Siri']['VehicleMonitoringDelivery']['VehicleActivity']['MonitoredVehicleJourney'])
df

,LineRef,DirectionRef,FramedVehicleJourneyRef,PublishedLineName,OriginRef,DestinationRef,Monitored,VehicleLocation,ProgressRate,CourseOfJourneyRef,VehicleRef,Extensions
0,703,None,"{'DataFrameRef': '2024-03-05T00:00:00-07:00', ...",RED LINE,0,0,True,"{'Longitude': '-111.890983', 'Latitude': '40.7...",1,400864,1102,"{'LastGPSFix': '2024-03-05T16:52:06.26', 'Sche..."
1,703,None,"{'DataFrameRef': '2024-03-05T00:00:00-07:00', ...",RED LINE,0,0,True,"{'Longitude': '-112.0144435', 'Latitude': '40....",1,400855,1105,"{'LastGPSFix': '2024-03-05T16:52:05.367', 'Sch..."
2,703,TO MEDICAL,"{'DataFrameRef': '2024-03-05T00:00:00-07:00', ...",RED LINE,TX101715,TX127252,True,"{'Longitude': '-112.01446', 'Latitude': '40.54...",1,367715,1105,"{'LastGPSFix': '2024-03-05T16:50:33.95', 'Sche..."
3,703,TO MEDICAL,"{'DataFrameRef': '2024-03-05T00:00:00-07:00', ...",RED LINE,TX101715,TX127252,True,"{'Longitude': '-111.8598765', 'Latitude': '40....",1,399562,1107,"{'LastGPSFix': '2024-03-05T16:52:11.43', 'Sche..."
4,703,None,"{'DataFrameRef': '2024-03-05T00:00:00-07:00', ...",RED LINE,0,0,True,"{'Longitude': '-111.852222', 'Latitude': '40.7...",1,400837,1132,"{'LastGPSFix': '2024-03-05T16:52:11.43', 'Sche..."
5,703,None,"{'DataFrameRef': '2024-03-05T00:00:00-07:00', ...",RED LINE,0,0,True,"{'Longitude': '-111.89100716666667', 'Latitude...",1,400794,1156,"{'LastGPSFix': '2024-03-05T16:52:06.117', 'Sch..."
6,703,None,"{'DataFrameRef': '2024-03-05T00:00:00-07:00', ...",RED LINE,0,0,True,"{'Longitude': '-111.972038', 'Latitude': '40.5...",1,400866,1167,"{'LastGPSFix': '2024-03-05T16:52:11.493', 'Sch..."


In [82]:
location_df = df[['VehicleLocation']]
location_df

,VehicleLocation
0,"{'Longitude': '-111.890983', 'Latitude': '40.7..."
1,"{'Longitude': '-112.0144435', 'Latitude': '40...."
2,"{'Longitude': '-112.01446', 'Latitude': '40.54..."
3,"{'Longitude': '-111.8598765', 'Latitude': '40...."
4,"{'Longitude': '-111.852222', 'Latitude': '40.7..."
5,"{'Longitude': '-111.89100716666667', 'Latitude..."
6,"{'Longitude': '-111.972038', 'Latitude': '40.5..."


In [83]:
# prompt: convert dict to columns in location_df

import pandas as pd
location_df = pd.json_normalize(location_df['VehicleLocation'])
location_df


,Longitude,Latitude
0,-111.890983,40.758957
1,-112.0144435,40.544429833333332
2,-112.01446,40.54449
3,-111.8598765,40.758508
4,-111.852222,40.759515
5,-111.89100716666667,40.7546115
6,-111.972038,40.594772


In [84]:
def get_locations():
    url = f'http://api.rideuta.com/SIRI/SIRI.svc/VehicleMonitor/ByRoute?route={route}&onwardcalls=true&usertoken={uta_secret}'
    response = requests.get(url)
    xml_dict = xmltodict.parse(response.text)
    df = pd.DataFrame(xml_dict['Siri']['VehicleMonitoringDelivery']['VehicleActivity']['MonitoredVehicleJourney'])
    location_df = pd.json_normalize(df['VehicleLocation'])
    return location_df

In [ ]:
# prompt: limit the folium map to a certain size

import folium

# Create a map object with the desired location and zoom level
m = folium.Map(location=[37.7749, -122.4194], zoom_start=12)

# Limit the map size to a specific width and height
m.get_root().html.add_child(folium.Element(f'<style>*{selector} {{width: {width}px; height: {height}px;}}</style>', {'selector': 'map'}))

# Display the map
m


In [107]:
import folium

# Create a base map centered at the specified latitude and longitude.
map = folium.Map(location=[40.7128, -111.85], zoom_start=12)

location_df = get_locations()

# Add a marker to the map at the specified latitude and longitude.
for (lat, lon) in zip(location_df['Latitude'], location_df['Longitude']):
    folium.Marker([lat, lon], popup='UTA Bus Stop').add_to(map)

# Display the map.
map
